### Imports

In [1]:
import sys
import os
import importlib
sys.path.insert(0, os.path.abspath("../../data_model/"))

In [2]:
import pandas as pd
import numpy as np
from pydantic import ValidationError
import data_model
import enums as e
from utils import extract_base_type, add_enum_label_columns, add_list_objects, add_synthetic_records, map_zones
import datetime

In [3]:
importlib.reload(data_model)
importlib.reload(e)
from data_model import Respondent, Employee, AirPassenger, Trip, DepartingPassengerResident, DepartingPassengerVisitor, ArrivingPassengerResident, ArrivingPassengerVisitor, DepartingAirPassenger, ArrivingAirPassenger, Resident, Visitor

### I/O

In [4]:
external_dir = "../../data/external"
interim_dir = "../../data/interim"
processed_dir = "../../data/processed"

In [5]:
clean_survey_file = os.path.join(interim_dir, "survey_data_clean.csv")
output_csv_filename = os.path.join(processed_dir, "data_model_output.csv")

respondents_to_not_model_file = os.path.join(processed_dir, "respondents_to_not_model.csv") #Responses to not weight

In [6]:
clean_df = pd.read_csv(clean_survey_file)

C:\Users\mwe\AppData\Local\Temp\ipykernel_72816\824925239.py:1: DtypeWarning: Columns (6,28,46,68,71,83,86,95,111,114,219,260,265,288,291,326,371) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_df = pd.read_csv(clean_survey_file)


### Select Variables to verify for the survey

The variables which need to be verified are extracted from the data model. 

In [7]:
respondent_variables = [field_name for field_name, field_info in Respondent.__fields__.items()]


trip_variables = [field_name for field_name, field_info in Trip.__fields__.items()]
trip_variables.append('respondentid')
#trip_variables.remove('valid_record')
trip_variables.remove('validation_error')
trip_variables.remove('validation_severity')
trip_variables.remove('validation_num_errors')

employee_variables = [field_name for field_name, field_info in Employee.__fields__.items()]
employee_variables.remove('trip')

#air_passenger_variables = [field_name for field_name, field_info in AirPassenger.__fields__.items()]
#air_passenger_variables.remove('trip')

air_passenger_departing_resident_variables = [field_name for field_name, field_info in DepartingPassengerResident.__fields__.items()]
air_passenger_departing_visitor_variables = [field_name for field_name, field_info in DepartingPassengerVisitor.__fields__.items()]
air_passenger_arriving_resident_variables = [field_name for field_name, field_info in ArrivingPassengerResident.__fields__.items()]
air_passenger_arriving_visitor_variables = [field_name for field_name, field_info in ArrivingPassengerVisitor.__fields__.items()]
air_passenger_variables = list(set(air_passenger_departing_resident_variables + air_passenger_departing_visitor_variables + air_passenger_arriving_resident_variables +air_passenger_arriving_visitor_variables))

variables_to_verify = list(set(air_passenger_variables + respondent_variables + trip_variables + employee_variables))
variables_to_verify.remove('trip')
#variables_to_verify.remove('valid_record')
variables_to_verify.remove('validation_error')
variables_to_verify.remove('validation_severity')
variables_to_verify.remove('validation_num_errors')

working_df = clean_df.copy()
working_df = working_df[variables_to_verify].copy()
working_df = working_df.loc[working_df['marketsegment'].notna()].copy()

C:\Users\mwe\AppData\Local\Temp\ipykernel_72816\1708849770.py:1: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  respondent_variables = [field_name for field_name, field_info in Respondent.__fields__.items()]
C:\Users\mwe\AppData\Local\Temp\ipykernel_72816\1708849770.py:4: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  trip_variables = [field_name for field_name, field_info in Trip.__fields__.items()]
C:\Users\mwe\AppData\Local\Temp\ipykernel_72816\1708849770.py:11: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Mi

In [8]:
working_df.shape

(5711, 324)

### Serialize the data

In [9]:
trips_df = working_df[trip_variables].copy()
persons_df = working_df[list[set(employee_variables + respondent_variables + air_passenger_variables)]].copy()

In [10]:
# combined
respondent_list = add_list_objects(
        trips_df.to_dict(orient="records"),  #child list
        "respondentid", # child key
        persons_df.to_dict(orient="records"), # parent list
        "respondentid", # parent key
        "trip", # parent var
    )

In [11]:
len(respondent_list)

5711

In [12]:
employee_list = []
arriving_air_passenger_resident_list = []
arriving_air_passenger_visitor_list = []
departing_air_passenger_resident_list = []
departing_air_passenger_visitor_list = []
other_list = []
failed_records = []

for respondent in respondent_list:
     market_segment = respondent["marketsegment"]
     try:
        if market_segment == e.Type.EMPLOYEE:
            ev = Employee(** respondent)
            employee_list.append(ev)
        elif market_segment == e.Type.PASSENGER:
             passenger_segment= respondent["passenger_segment"]
             if passenger_segment == e.PassengerSegment.RESIDENT_ARRIVING:
                    apr = ArrivingPassengerResident(** respondent)
                    arriving_air_passenger_resident_list.append(apr)
             elif passenger_segment == e.PassengerSegment.VISITOR_ARRIVING:
                    apv = ArrivingPassengerVisitor(** respondent)
                    arriving_air_passenger_visitor_list.append(apv)
             elif passenger_segment == e.PassengerSegment.RESIDENT_DEPARTING:
                    dpr = DepartingPassengerResident(** respondent)
                    departing_air_passenger_resident_list.append(dpr)
             elif passenger_segment == e.PassengerSegment.VISITOR_DEPARTING:
                    dpv = DepartingPassengerVisitor(** respondent)
                    departing_air_passenger_visitor_list.append(dpv)
             else:
                    rv = Respondent(** respondent)
                    other_list.append(rv)

        else:
            rv = Respondent(** respondent)
            other_list.append(rv)

     except ValidationError as err:
            respondent['error_flag'] = 'failed'
            respondent['error_message'] = str(err)
            failed_records.append(respondent)


failed_df = pd.DataFrame(failed_records)
failed_df.head()

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='40', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='5', input_type=str])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='40', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='5', inp

Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: nights_visited
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed fo

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='90', input_type=str])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='90', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [input_value=<Terminal.TERMINAL_1: 1>, input_type=Terminal])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025

Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critica

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='27', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='2', input_type=str])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='27', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='2', inp

...

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='22.0', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='8.0', input_type=str])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='22.0', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='8

Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: trip_start_time
Critical check failed for variable: main_mode
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: nights_visited
Critical check failed for variable: main_mode
Critical check failed for variable: main_mode
Critical check failed for variable: trip_arrival_time
Critical check failed for variable: trip_arrival_time
Critical check failed for variable: reverse_mode
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: nights_visited
Critical check failed for variable: main_mode
Critical check failed for variable: main_mode
Critical check failed for variable: main_mode
Critical check fail

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=['main_mode: Critical', '...in_longitude: Critical'], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [input_value=<Terminal.TERMINAL_1: 1>, input_type=Terminal])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=['main_mode: Critical', '...on_longitude: Critical'], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [input_value=<Terminal.UNKNOWN: 99

Critical check failed for variable: trip_arrival_time
Critical check failed for variable: trip_arrival_time
Critical check failed for variable: reverse_mode
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: nights_visited
Critical check failed for variable: main_mode
Critical check failed for variable: main_mode
Critical check failed for variable: main_mode
Critical check failed for variable: main_mode
Critical check failed for variable: trip_arrival_time
Critical check failed for variable: trip_arrival_time
Critical check failed for variable: reverse_mode
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: nights_visited
Critical check failed for variable: main_mode
Critical check failed for variable: main_mode
Critical check failed for variab

,general_use_transit_visitor_home,sp_other_airport_cost,reasons_no_transit_ride_too_long,sp_feature_short_wait,egress_mode_other,from_airport_transit_route_2,sdia_transit_awareness,destination_name,flight_purpose_other,sdia_accessmode_split_hotel_shuttle_van,...,reasons_no_transit_not_safe,sp_feature_seats_transit,to_airport_transit_route_1,sp_other_airport_ease_of_boarding,number_of_travel_companions,sp_feature_weekend_frequency,taxi_fhv_fare,trip,error_flag,error_message
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},failed,68 validation errors for ArrivingPassengerVisi...


In [13]:
failed_df.shape

(1, 327)

In [14]:
#failed_df['error_message'].unique()

In [15]:
#failed_df['is_completed'].value_counts()

In [16]:
#failed_df['error_message'][0]

In [17]:
failed_df.to_csv(f'{processed_dir}/failed_records.csv', index = False)

In [18]:
len(failed_df)

1

In [19]:
print("Arriving Air Passengers Residents:", len(arriving_air_passenger_resident_list))
print("Arriving Air Passengers Visitors:", len(arriving_air_passenger_visitor_list))
print("Departing Air Passengers Residents:", len(departing_air_passenger_resident_list))
print("Departing Air Passengers Visitors:", len(departing_air_passenger_visitor_list))
print("Total Air Passengers:", len(arriving_air_passenger_resident_list) + len(arriving_air_passenger_visitor_list) + len(departing_air_passenger_resident_list) + len(departing_air_passenger_visitor_list))

Arriving Air Passengers Residents: 73
Arriving Air Passengers Visitors: 168
Departing Air Passengers Residents: 1795
Departing Air Passengers Visitors: 2726
Total Air Passengers: 4762


### Make Data

In [20]:
employee_df = pd.DataFrame([Employee.model_dump() for Employee in employee_list])

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='100', input_type=str])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type

In [21]:
#passenger_df = pd.DataFrame([AirPassenger.model_dump() for AirPassenger in air_passenger_list])

In [22]:
arriving_passenger_resident_df = pd.DataFrame([ArrivingPassengerResident.model_dump() for ArrivingPassengerResident in arriving_air_passenger_resident_list])
arriving_passenger_visitor_df = pd.DataFrame([ArrivingPassengerVisitor.model_dump() for ArrivingPassengerVisitor in arriving_air_passenger_visitor_list])
departing_passenger_resident_df = pd.DataFrame([DepartingPassengerResident.model_dump() for DepartingPassengerResident in departing_air_passenger_resident_list])
departing_passenger_visitor_df = pd.DataFrame([DepartingPassengerVisitor.model_dump() for DepartingPassengerVisitor in departing_air_passenger_visitor_list])

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=['reverse_mode: Non-Critical'], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [input_value=<Terminal.TERMINAL_1: 1>, input_type=Terminal])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=['party_size_ground_acces...rse_mode: Non-Critical'], input_type=list])
  PydanticSerializatio

In [23]:
other_df = pd.DataFrame([Respondent.model_dump() for Respondent in other_list])
# other_df = add_enum_label_columns(other_df, Respondent)

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=['main_mode: Critical', '...in_longitude: Critical'], input_type=list])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=['main_mode: Critical'], input_type=list])
  return self.__pydantic_serializer__.to_python(


In [24]:
len(other_list)

192

In [25]:
trip_list = []
id_list = []
for record in employee_list + arriving_air_passenger_resident_list + arriving_air_passenger_visitor_list + departing_air_passenger_resident_list + departing_air_passenger_visitor_list  + other_list:
    trip_list.append(record.trip)
    id_list.append(record.respondentid)

trip_df = pd.DataFrame([Trip.model_dump() for Trip in trip_list])
id_df = pd.DataFrame(id_list, columns=["respondentid"])

trip_df = pd.concat([id_df, trip_df], axis=1)
trip_df = add_enum_label_columns(trip_df,Trip)


t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [input_value='100', input_type=str])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[], input_type=list])
  return self.__pydantic_serializer__.to_python(
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\.venv\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` -

In [26]:
output_df = pd.concat([employee_df, arriving_passenger_resident_df, arriving_passenger_visitor_df, departing_passenger_resident_df, departing_passenger_visitor_df , other_df], axis=0).reset_index(drop=True).drop(columns=["trip"])
output_df = pd.merge(output_df, trip_df, on="respondentid", how="left", suffixes = ('_person', '_trip'))

C:\Users\mwe\AppData\Local\Temp\ipykernel_72816\280131642.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([employee_df, arriving_passenger_resident_df, arriving_passenger_visitor_df, departing_passenger_resident_df, departing_passenger_visitor_df , other_df], axis=0).reset_index(drop=True).drop(columns=["trip"])


In [27]:
output_df.shape

(5710, 357)

In [28]:
#Consolidated validation check field
output_df['is_valid_record'] = (output_df['initial_etc_check'] == True) & (output_df['validation_severity_person']!='Critical') & (output_df['validation_severity_trip'] != 'Critical') & (output_df['thanksgiving_week_flag'] == False)

In [29]:
output_df.loc[~output_df['is_valid_record'], 'weight'] = 0


In [30]:
output_df = add_synthetic_records(output_df)

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\data_model\utils.py:272: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df, synthetic_df], ignore_index=True)


In [31]:
# output_df.head()

In [32]:
output_df =  add_enum_label_columns(output_df, Respondent)

output_df = add_enum_label_columns(output_df, AirPassenger)
output_df = add_enum_label_columns(output_df, DepartingAirPassenger)
output_df =  add_enum_label_columns(output_df, ArrivingAirPassenger)
output_df =  add_enum_label_columns(output_df, Resident)
output_df =  add_enum_label_columns(output_df, Visitor)

output_df = add_enum_label_columns(output_df, DepartingPassengerResident)
output_df =  add_enum_label_columns(output_df, ArrivingPassengerResident)

output_df = add_enum_label_columns(output_df, DepartingPassengerVisitor)
output_df =  add_enum_label_columns(output_df, ArrivingPassengerVisitor)

output_df =  add_enum_label_columns(output_df, Trip)
output_df =  add_enum_label_columns(output_df, Employee)

t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\data_model\utils.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enum_name_col] = df[field].map(enum_names).astype(str)
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\data_model\utils.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enum_name_col] = df[field].map(enum_names).astype(str)
t:\Survey\SDIA\2023\Survey_Scripts_May31_2025\Survey_Airport\data_model\utils.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually

In [33]:
# Define the list of important columns
important_columns = ['respondentid', 'is_completed', 'is_valid_record', 'date_completed','time_completed', 'is_pilot', 'is_self_administered', 'record_type_synthetic']

# Separate important columns and the rest of the columns
remaining_columns = [col for col in output_df.columns if col not in important_columns]

# Create the new column order
new_column_order = important_columns + sorted(remaining_columns)

# Reorder the DataFrame
output_df = output_df[new_column_order]

In [34]:
# introduce column that flags respondents to be modeled by SANDAG ECON
respondents_to_not_model = pd.read_csv(respondents_to_not_model_file, usecols=['respondentid'])['respondentid'].tolist()

output_df.loc[
        output_df['record_type_synthetic'] == False,
        'model_respondent_bool'
    ] = (
        ~
        output_df
        .query('record_type_synthetic == False')
        ['respondentid']
        .fillna(0)
        .astype(int)
        .isin(respondents_to_not_model)
        )

C:\Users\mwe\AppData\Local\Temp\ipykernel_72816\330081304.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)
C:\Users\mwe\AppData\Local\Temp\ipykernel_72816\330081304.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df.loc[


In [35]:
output_df.index = output_df.index + 1
output_df.to_csv(output_csv_filename, index_label = 'unique_id')